# Introduction
**Deep learning approaches** (sub-symbolic) are capable processing high dimensional data like images much better than any previous technique.
They do however have several drawbacks, a major of which is that models are very hard to analyze, making it difficult to understand why the system behaves as it does, how one change will affect the system as a whole, or debug any issues that might arise.
Traditional **logic based approaches** to artificial intelligence (symbolic) are bad at dealing natural data like images, but the internal mechanisms are much easier to understand.
Logic based approaches also have advantages in some domains, for instance when long term planning is involved.

Symbolic and sub-symbolic approaches are largely incompatible due to the lack of a coherent way to interface between them [4].
Since the strengths and weaknesses of these fields compliment each other so well such an interface is of great practical and scientific importance.
Asai and Fukunaga [4] showed that it was possible to combine a neural network model and a classical planning algorithm via the use of discrete representations.
In their model they used one network to learn a discrete embedding of the observations, and another network to learn the dynamics of the environment, updating the discrete embedding given an action.
The key contribution is that these categorical variables can be used directly as propositional symbols by a symbolic reasoning system.
They show this by using a classical planning algorithm to make an action plan using the discrete embeddings and the model of the dynamics.
Using this approach they were able to solve some simple puzzles (8-puzzel, tower of Hanoi) that had been modified such that they required a vision system in order to understand the state of the environment.
The contribution of Asai and Fukunaga [4] is an important first step towards combining symbolic and sub-symbolic approaches.
The problems tackled are however very simple and how well the proposed methods scale to more difficult problems remains to be seen.


Ha and Schmidhuber [3] proposed a simple yet powerfull framework, 'World Models', that have several similarities with the one proposed by Asai and Fukunaga [4].
World Models also use a deep neural network to learn an embedding of the environment, and another to learn the dynamics in order to update the embeddings.
Using the emedding and hidden states of the dynamics network they were able to train a simple linear agent.
Ha and Schmidhuber [3] were able to show that the World Model framework is capable of training large networks, and solve complicated environments (OpenAI CarRacing, ViZDoom).

> ![](images/CarRacing.png)
>
> Example frame from the CarRacing environment. In CarRacing the objective is to control (accelerate, turn, break) the red car, such that it drives along as much of the track as possible in a given amount of time.
> A new randomly generated track is created every time.

The key difference between the two is that Ha and Schmidhuber [3] uses a continuous latent space, making it incompatible with classical planning algorithms, and that they are capable of training much larger networks, capable of handling much more complicated problems.


In this notebook we take a small step towards combining the work by Asai and Fukunaga [4] and Ha and Schmidhuber [3].
Specifically in this notebook we train a network to learn a discrete representation of the [OpenAI CarRacing](https://gym.openai.com/envs/CarRacing-v0/) environment, and train a simple linear agent to control the car.

## World Models
The World Models framework proposed by Ha and Schmidhuber [3] is an example of *model based reinforcement learning* i.e. a model of the enironment is learnt, which is then used to train the agent (in part or fully).
Model based reinforcement learning has many advantages[1, 2], for instance:

* **Prior knowledge** of the environment can be encoded more easily than in model-free methods.
* **Transfer learning** between tasks for the same environment is easily facilitated through models.
* **High-dimentional observations** introduce several complexitites (curse of dimensionality) making it harder to solve a task. Models can typically provide low-dimensional representations that simplify the problem greatly.
* **Data efficent learning:** Model based approaches are generally much more data effcient that model-free, which makes a big difference when data is expensive to obtain as it limits the interactions with the true environment, potenitally making training safer and faster when the environment is physical or requires complex computations.(e.g. robots).
* **Internal Simulator** makes it possible to rollout several scenarios before making an action - enabling the use of methods like Monte Carlo tree search. This also helps with planning through by making long-term predictions through rollouts.


The World Models framework consists of three components that work closely together: 

* **Vision (V):** A convolutional variational auto-encoder with a Gaussian latent space ($z$).
* **Memory (M):** An auto-regressive model, implemented as an LSTM.
* **Controller (C):** A simple linear controller.

> ![Worldmodel diagram](images/worldmodel.png)
>
> Figure 4 from [3]

This setup with distinct modules enables V and M to be trained separately in a fully unsupervised way from data collected from a random policy.
This training setup provides rich training signals, enabling the use of large networks for RL, something that typically isn't possible as the reward is a relatively weak training signal.
Since C is just a linear model we can see that most of the complexity must reside in V and M.

Ha and Schmidhuber [3] also show that it possible to train the agent entirely within hallucinated environments (V and M, rather than the actual environment) and transfer the learnt policy directly to the real environment.

The 'World Models' framework that Ha and Schmidhuber [3] proposes combines several deep learning and reinforcement learning techniques into a simple architecture that achieve very good results.
Further more it is possible to train the V and M models in less than an hour on a normal GPU, speeding up development and total training time significantly.

# Experiments
In order to implify experiments we implement a simplified version the World Model, using only the vision module (V) and the linear controller (C).
The following sections describe the experimental setup and demonstrates our results.


## The Vision Module (V)

As for the architecture we used the same as Ha and Schmidhuber [4], i.e. a 4 layer convolutional encoder, a 32 dimensional latent space, and a 4 layer deconvolutional decoder.
For all activations, except the last one and the ones in the sampling layer, are ReLU.
No dropout or batch normalization was used.

> ![](images/vae.png)
>
> Modified from Ha and Schmidhuber [4].

For the continuous latent representation we use the standard reparametrization trick, as proposed by Kingma et al. [7].
Ind the discrete case we *Gumbel-softmax reparametrization trick* [5, 6], similar to Asai and Fukunaga [4].
In both cases the latent space is 32 dimensional, with each dimension having 32 bits of information.
In the continuous case each dimension is encoded as a 32 bit float, where as the discrete case is encoded as a 32 bit vector.


The loss functions are in both cases straight forward:
$$
L = \text{mse}(x, d(e(x))) + \alpha \text{KL}(p(z), e(x))
$$

where $\text{mse}$ is the _mean squared error_, $d(\cdot)$ is the decoder function, $e(\cdot)$ is the decoder function, $\text{KL}$ is the Kullback Leibler divergence, and $p(z)$ is the prior on the latent space i.e. a standard normal and uniform distribution for the continuous and discrete case respectively.
$\alpha$ is a weighting term that determines the influence of the KL term. 
In the continuous case this was keept constant at 1.
In the discrete case however we found it benifical to start it at 2, and exponentially anneal it to 0.01 with a half-time of 10 epochs.

Another important parameter is $\tau$, which is used in the Gumbel-softmax reparametrization trick to determine the degree of discreteness during training.
We started it at 5 and exponentially annealed it to 0.1 with a half-time of 10 epochs.


### Vision Module Training Implementation
For training the VAEs  we generated 256 random policy rollouts, each of length 300.
This is done using the `generate_VAE_data.py` script.
The continuous and discrete variational auto-encoders are trained using the scripts `train_agents.py` and `train_Gumbel_VAE.py` respectively.

The code below of load the validation data set, consiting of 16 random policy sequences of length 150, and performs some visualizations of it, the reconstruction, and the latent space.

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import numpy as np
from IPython import display
import matplotlib.pyplot as plt
import scipy.stats

import train_Gumbel_VAE
import train_VAE 
import visualize_VAE

In [ ]:
from skimage.transform import resize

# Load and visualize data
print("Loading data ...")
data96 = np.load('test_data.npy')
data64 = resize(data96, (data96.shape[0], 64, 64, 3), anti_aliasing=True, mode='reflect')

max_len = data64.shape[0]
num_batches = 16
eps_len = 150
data64_re = np.reshape(data64, [num_batches, eps_len, 64, 64, 3])

print('\tLoading complete.')
print('data shape:', data64.shape)

In [ ]:
# Animate a trajectory, examplifying the data.
eps = 11  # just an interesting episode

for i in range(eps_len):
    display.clear_output(wait=True)
#     plt.imshow(data64[i])
    plt.imshow(data64_re[eps][i])
    plt.title('t='+str(i))
    plt.show()

In [ ]:
# Compute predictions and latent space representations
print("Generating Continuous predictions")
sess_C, model_C = train_VAE.load_vae()
pred_C, mu, sigma, z = model_C.predict(sess_C, data96)
sess_C.close()

print("\nGenerating Discrete predictions")
sess_G, model_G = train_Gumbel_VAE.load_vae()
pred_G = model_G.model.predict(data64, verbose=0)
logits, pre_gumbel_softmax, gumbel, hard_sample = model_G.encoder([data64])
sess_G.close()

print("\nDone")

In [ ]:
# Define some helper plotting functions
def plot_continuous(start=0, stop=np.inf):
    for n in range(start, np.minimum(max_len, stop)):
        plt.figure(figsize=(9, 3))

        plt.subplot(131)
        plt.imshow(data64[n])
        plt.title('data, t=' + str(n-start))

        plt.subplot(132)
        plt.imshow(np.reshape(z[n], [4, 8]), cmap='hot', vmin=-5, vmax=5)
        plt.colorbar()
        plt.title('z')

        plt.subplot(133)
        plt.imshow(pred_C[n])
        plt.title('pred - Continuous')

        display.clear_output(wait=True)
        plt.tight_layout()
        plt.show()


def plot_discrete(start=0, stop=np.inf):
    N = model_G.N
    M = model_G.M
    for n in range(start, np.minimum(max_len, stop)):
        plt.figure(figsize=(9, 9))

        plt.subplot(321)
        plt.imshow(data64[n])
        plt.title('data, t=' + str(n-start))
        plt.subplot(322)
        plt.imshow(pred_G[n])
        plt.title('pred - Discrete')

        plt.subplot(323)
        plt.imshow(np.reshape(logits[n], (N, M)), cmap='hot')
        plt.title('logits')
        plt.colorbar()

        plt.subplot(324)
        plt.imshow(np.reshape(pre_gumbel_softmax[n], (N, M)), cmap='hot', vmin=0, vmax=1)
        plt.title('pre_gumbel_softmax')
        plt.colorbar()

        plt.subplot(325)
        plt.imshow(np.reshape(gumbel[n], (N, M)), cmap='hot', vmin=0, vmax=1)
        plt.title('gumbel')
        plt.colorbar()

        plt.subplot(326)
        plt.imshow(np.reshape(hard_sample[n], (N, M)), cmap='hot', vmin=0, vmax=1)
        plt.title('hard sample')
        plt.colorbar()

        display.clear_output(wait=True)
        plt.tight_layout()
        plt.show()

In [ ]:
# Plot continuous reconstructions and latent representations
start = eps*eps_len
stop = eps*eps_len+eps_len
print('start', start, '- stop', stop)
plot_continuous(start, stop)

In [ ]:
# Plot discrete reconstructions and latent representations
plot_discrete(start, stop)

### Results: Reconstructions

Both the continuous and discrete auto-encoder quickly learn make some reconstructions that capture the important aspects of the environment, such as what the road looks like, and where the car is in relation to the road.
Since a pixelwise loss is used the reconstructions are as expected somewhat blurry.
The discrete auto-encoder does however tend to have a slightly lower reconstruction error, as shown by the graph below.
The standard deviations do however overlap, so the difference isn't big.

Qualitatively we can see that the discrete VAE tends to have crisper edges, and more consistent predictions (i.e. the difference from frame to frame is smaller)

In [ ]:
def pred_error(true, pred):
    means = np.mean(np.abs(true - pred), axis=(2, 3, 4))
    std = np.std(means, axis=0)
    mean = np.mean(means, axis=0)
    return mean, std

pred_C_re = np.reshape(pred_C, [16, 150, 64, 64, 3])
pred_G_re = np.reshape(pred_G, [16, 150, 64, 64, 3])
c_mean, c_std = pred_error(data64_re, pred_C_re)
d_mean, d_std = pred_error(data64_re, pred_G_re)

plt.plot(c_mean.T, c='r', label='Continuous error mean')
plt.plot(c_mean-c_std, c='r', alpha=0.25, label='Continuous error +/- std')
plt.plot(c_mean+c_std, c='r', alpha=0.25)

plt.plot(d_mean.T, c='b', label='Discrete error')
plt.plot(d_mean-d_std, c='b', alpha=0.25, label='Discrete error +/- std')
plt.plot(d_mean+d_std, c='b', alpha=0.25)
plt.legend()
plt.show()

### Results: Representations
Another way to examine the quality of of the VAE is to examine how the KL divergence between the posterior at time $t$ and time $t+1$ differ.

$$
\text{KL}(e(x_{t}), e(x_{t+1}))
$$
where again $e(\cdot)$ is the encoder function.

If the KL divergence between the embedding of two images is small it means that the images lie close to each other in embedding space.
If the frame-by-frame KL divergence is small for a naturally occuring sequence it means that the embedding space is structured such that observations that are close to each other in reality are close in embedding space, which is a desirable attribute.
We can compare a naturally occuring sequence with a random pertubation of it, in order to examine the structure of the embedding space.

In [ ]:
# Functions for computing KL divergence
def KL_seq_continuous(mu, sigma):
    def calc_kl(mu1, sigma1, mu2, sigma2):
        return np.log(sigma2, sigma1) + (sigma1**2 + (mu1-mu2)**2)/(2*sigma2**2) - 1/2
    kl_seq = []
    
    for i in range(mu.shape[0]-1):
        kl = calc_kl(mu[i],sigma[i], mu[i+1],sigma[i+1])
        kl_seq.append(kl)

    kl_seq = np.mean(np.array(kl_seq), 1)
    return kl_seq

def KL_seq_discrete(probs):
    kl_seq = []
    for i in range(probs.shape[0]-1):
        kl_seq.append(0)
        for j in range(probs.shape[1]):
            ent = scipy.stats.entropy(probs[i,j] + 1e-30, probs[i+1,j] + 1e-30)
            kl_seq[-1] += ent
        kl_seq[-1] /= probs.shape[1]

    return np.array(kl_seq)

In [ ]:
# Plot of KL divergence 

# Plot continuous
mu_ = np.copy(mu[eps*eps_len:(eps+1)*eps_len])
sigma_ = np.copy(sigma[eps*eps_len:(eps+1)*eps_len])
permuation = np.random.permutation(len(mu_))
kl_cont = KL_seq_continuous(mu_, sigma_)

mu_ = np.copy(mu[permuation])
sigma_ = np.copy(sigma[permuation])
kl_cont_shuffle = KL_seq_continuous(mu_, sigma_)

plt.plot(kl_cont_shuffle, label='Random')
plt.plot(kl_cont, lw=3, label='Ordered')
plt.title('KL for sequence - Continuous')
plt.legend()
plt.ylim([-5,100])
plt.show()

# Plot discrete
pre_gumbel_softmax_ = pre_gumbel_softmax[eps*eps_len:(eps+1)*eps_len]
kl_disc = KL_seq_discrete(pre_gumbel_softmax_)
kl_disc_shuffle = KL_seq_discrete(pre_gumbel_softmax_[permuation])
plt.plot(kl_disc_shuffle, label='Random')
plt.plot(kl_disc, lw=3, label='Ordered')

plt.title('KL for sequence - Discrete')
plt.legend(loc=1)
plt.show()

In the continuous case we see that the ordered sequence has a nice smoothly varying value with three peaks: one at $0$, one at $20$ and one at $110$.
The random sequence is much more jagged, with a few extreme spikes.
The numberical value between the ordered and random sequence is, with a few exceptions, quite close, suggesting that the embedding space is not that structured.
Examining the peaks we see that the peak at $0$ is because CarRacing starts zoomed out, and zooms in rapidly over the first couple of frames.
The other two peaks do however not have any clear interpretation - i.e. there are no sudden changes in the images, further suggesting that the embedding lacks structure.


Looking at the discrete case we see a for the ordered sequence that the KL divergence starts high, and quickly drops to a low value and stays there for the duration of the sequence.
On the otherhand the random sequence has very high KL values, and is clearly distinguishable from the ordered sequence.

This suggests that there is more structure in the discrete embedding space than in the continuous one.

## The Controller (C)

For the controller we use a simple linear controller, similar to the one used by Ha and Schmidhuber [4].
For each action output is simply a weighted linear combination of the input followed by a $tanh$ squeezing function to make sure the actions are in the correct ranges, i.e.

$$
a_t = tanh(W*z_t + b)
$$

In the continuous case the VAE is used exactly as during training, but for the discrete VAE a hard sample was performed from the pre-Gumbel softmax values.

One simplification was made to the environment.
Traditionally there are three continuous actions: steering left/right, acceleration, and brake.
In order to speed up training it was simplified such that breaking and acceleration were combined into one variable, with positive numbers representing acceleration and negative breaking.


### Controller Training Implementation
The agents are trained using `train_agents.py` and `train_agents.py`

The agent trained on a continuous latent space has a total of $32*2+2=66$ parameters, whereas the agent trained on the discrete has $32*32*2+2=2050$.

The weights were learned using *Covariance Matrix Adaptation Evolution Strategy* (CMA-ES), similar to Ha and Schmidhuber [4].
For our experiments each generation had a population size of 12, with the fitness of each agent being the average cumulative reward over 8 trials with different random seeds.

Below are the learning curves for both the continuous and discrete agent plotted:

In [ ]:
from display_rewards import display_rewards

file_names = ['rewards_continuous.npy', 'rewards_gumbel.npy']
display_rewards(file_names)

We see that the continuous agent learns both quicker and achieves a higer toal than the discrete.
It is also not clear whether the agents have fully converged at the time training ended, so we do not know which agent would ultimately achieve the highest score if training was allowed to continue.
The slower training time of the discrete agent is not surprising, given that it has many more parameters (~ a factor 31) than the continuous one.
CMA-ES was probably a great choice of optimization algorithm for this project, as it suffers greatly when the search space becomes large.
Further more it is unable to take advantage of the discreteness of the signals in the discrete case. 
A gradient based algorithm would be able to take advantage of the sparsity of the signals, leading to the same number of parameter updates per time step per agent. 

During training it was noticed that the discrete agent would frequently '*get stuck*' in place, without ever accelerating, something that wasn't noticed in the continuous case.
This might be, in part due to the low stochasticity in the system, suggesting possibly that the KL term was annealy to low. 
But it might also be an artifact due to the discrete embedding.
By having a separate weight for each bit each weight can be more specialized, which might increase the risk of unwanted behavior.
We cannot determine this from our experiments, but it is an interesting topic for future research.


Both the continuous and discrete agent achieve considerably worse scores than those optained by Ha and Schmidhuber [4], who manages to achieve an average of $632 \pm 251$ over 100 trials in a similar setup.
We expect that the main reason for our worse performance is training hyperparameters and lack of training time.
Ha and Schmidhuber [4] used a population size of 64 and computed the fitness over 16 episodes, which they ran for 2000 generations.
For our experiments we did not have access to suficient computational resources to allow for such training regiments.


# Conclusion

First we trained both a continuous and discrete variational auto-encoder (VAE).
Both were able to learn reconstructions that were able to capture important features of the environment, such as the shape of road using the same amount of information in the bottle neck ($32*32=1024$ bits).
The discrete VAE was able to achieve a lower reconstruction error, and the embedding also displayed more signs of structure than was the case for the continuous VAE.
These results confirm that discrete embeddings are an interesting and useful alternative to their continuous counterparts.
The images examined here are ofcourse very simple, so more work needs to be done on more complex cases.


The second part of the experiments focused on training a linear agent to play the CarRacing environment.
We were unable to achieve results on par with those by Ha and Schmidhuber [4], both in the continuous case, and the discrete, most likely due to the small computational resources we had available.
This diminishes the amount for information that can be read into the results.
This work does however spark an interesting question: How does one best take advantage of discrete representations?


# Aknowledgements
For this project several online resources were used inspiration that greatly speed up development of this project.
In particular the Github repository *dariocazzani/World-Models-TensorFlow* was used as a starting point for the code used for these experiments.

* https://github.com/dariocazzani/World-Models-TensorFlow
* https://medium.com/applied-data-science/how-to-build-your-own-world-model-using-python-and-keras-64fb388ba459
* https://github.com/AppliedDataSciencePartners/WorldModels
* https://github.com/hardmaru/WorldModelsExperiments


# Refrences
[1] Shakir Mohamed (2018) A Case Against Generative Models in RL?. Lecture at DALI 2018 Workshop on Generative Models in Reinforcement Learning. https://www.youtube.com/watch?v=EA2RtXsLSWU

[2] Chelsea Finn (2017) Deep RL Bootcamp Lecture 9 Model-based Reinforcement Learning. Lecture at Deep RL Bootcamp. https://www.youtube.com/watch?v=iC2a7M9voYU&feature=youtu.be

[3] Ha and Schmidhuber, "World Models", 2018. https://doi.org/10.5281/zenodo.1207631 https://worldmodels.github.io

[4] Asai, M., & Fukunaga, A. (2017). Classical Planning in Deep Latent Space: Bridging the Subsymbolic-Symbolic Boundary. Retrieved from http://arxiv.org/abs/1705.00154, 

[5] Chris J. Maddison, Andriy Mnih, and Yee Whye Teh. “The Concrete Distribution: a Continuous Relaxation of Discrete Random Variables.” ICLR Submission, 2017.

[6] Eric Jang, Shixiang Gu and Ben Poole. “Categorical Reparameterization by Gumbel-Softmax.” ICLR Submission, 2017.

[7] Kingma, Diederik P and Welling, Max. "Auto-Encoding Variational Bayes" International Conference on Learning Representations, 2014.